# Исследование текущего уровня потребительской лояльности

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила нас подготовить дашборд с его итогами. 

Чтобы оценить результаты опроса, оценки поделили на три группы:

    9-10 баллов — «cторонники» (англ. promoters);
    7-8 баллов — «нейтралы» (англ. passives);
    0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывали по формуле: % «сторонников» - % «критиков».

**План работы:**

    1. Подключится к базе данных
    2. Подготовить данные для дашборда, используя только SQL-запросы
    3. Построить дашборды в Tableau для ответа на вопросы:
    3.1. Как распределены участники опроса по возрасту и полу? 
    3.2. Каких пользователей больше: новых или старых? 
    3.3. Пользователи из каких городов активнее участвовали в опросе?
    3.4. Какие группы пользователей наиболее лояльны к сервису? Какие менее?
    3.5. Какой общий NPS среди всех опрошенных?
    3.6. Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?
    4. Опубликовать дашборд на сайте Tableau Public 

In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
    SELECT u.user_id,
           u.lt_day,
           n.is_new,
           u.age,
           CASE 
               WHEN u.gender_segment = 0 THEN 'мужчина' 
               WHEN u.gender_segment = 1 THEN 'женщина'
               ELSE 'не указано' 
           END AS gender_segment,
           u.os_name,
           u.cpe_type_name,
           l.country,
           l.city,
           SUBSTRING(ag.title, 4) AS age_segment,
           SUBSTRING(tr.title, 4) AS traffic_segment,
           SUBSTRING(lf.title, 4) AS lifetime_segment,
           u.nps_score,
           nps.nps_group
    FROM user AS u
    LEFT OUTER JOIN location AS l ON u.location_id = l.location_id
    LEFT OUTER JOIN age_segment AS ag ON u.age_gr_id = ag.age_gr_id
    LEFT OUTER JOIN traffic_segment AS tr ON u.tr_gr_id = tr.tr_gr_id
    LEFT OUTER JOIN lifetime_segment AS lf ON u.lt_gr_id = lf.lt_gr_id
    LEFT OUTER JOIN (SELECT user_id,
                            CASE 
                                 WHEN lt_day > 365 THEN 'опытный'
                                 ELSE 'новичок'
                            END AS is_new
                     FROM user) AS n ON u.user_id = n.user_id
    LEFT OUTER JOIN (SELECT user_id,
                            CASE 
                                 WHEN nps_score >= 9 THEN 'cторонники'
                                 WHEN nps_score BETWEEN 7 AND 8 THEN 'нейтралы'
                                 ELSE 'критики'
                            END AS nps_group
                     FROM user) AS nps ON u.user_id = nps.user_id;"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,опытный,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,опытный,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,опытный,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Презентация: https://public.tableau.com/views/_17266873515940/sheet17?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link